In [38]:
import sys
import pathlib

sys.path.append(r"C:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition")

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

IMAGE_SIZE = 64
BATCH_SIZE = 64
DATASET_REPETITIONS = 5
LOAD_MODEL = False

NOISE_EMBEDDING_SIZE = 32
PLOT_DIFFUSION_STEPS = 20

# optimization
EMA = 0.999
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50


In [40]:
checkpoints_folder = pathlib.Path.cwd()/"torch_checkpoints"
print(f"checkpoints folder : {checkpoints_folder} and it exists {checkpoints_folder.exists()}")
images_folder = pathlib.Path.cwd()/"output"/"generated_images_torch"
print(f"images folder : {images_folder} and it exists : {images_folder.exists()}")

checkpoints folder : c:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\notebooks\08_diffusion\01_ddm\torch_checkpoints and it exists True
images folder : c:\Users\amrul\programming\deep_learning\dl_projects\Generative_Deep_Learning_2nd_Edition\notebooks\08_diffusion\01_ddm\output\generated_images_torch and it exists : True


In [41]:
from notebooks.utils import display

In [42]:
import torch
import torch.nn as nn
from ddm_torch_model import DiffusionModel,get_flower_images_train_dataset,convert_images_torch_to_numpy_for_display,training_loop,Normalizer

In [44]:
train_data = get_flower_images_train_dataset(should_normalize=True)
print(f"Loaded flower train dataset : {len(train_data)}")

Loaded flower train dataset : 6552


In [45]:
image,label=train_data[10]

In [47]:
image[0,:].std()

tensor(1.2872)

In [7]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)

In [8]:
def calculate_mean_std(loader):
    sum_mean = 0.0
    sum_variance = 0.0
    total_images = 0

    for images, _ in loader:
        batch_images = images.size(0)
        images = images.view(batch_images, images.size(1), -1)
        sum_mean += images.mean(2).sum(0)
        sum_variance += ((images - images.mean(2, keepdim=True))**2).sum(2).sum(0)
        total_images += batch_images

    mean = sum_mean / total_images
    variance = sum_variance / total_images
    std = torch.sqrt(variance)
    return mean, std


In [9]:
# # Function to calculate mean and std
# def calculate_mean_std(loader):
#     mean = 0.0
#     std = 0.0
#     total_images = 0
#     for images, _ in loader:
#         batch_images = images.size(0)
#         images = images.view(batch_images, images.size(1), -1)
#         mean += images.mean(2).sum(0)
#         std += images.std(2).sum(0)
#         total_images += batch_images

#     mean /= total_images
#     std /= total_images
#     return mean, std

In [10]:
mean, std = calculate_mean_std(train_loader)
print(f"mean : {mean}, std : {std}")

mean : tensor([0.4353, 0.3773, 0.2871]), std : tensor([16.6121, 13.1927, 14.0779])


In [13]:
image,label = train_data[10]

In [15]:
image2,label2=train_data[20]

In [32]:
# concatenate all images in train_data
images=torch.cat([image.unsqueeze(0) for image,label in train_data],dim=0)

In [33]:
# at this point images have a shape of (N,C,H,W)
images.shape

torch.Size([6552, 3, 64, 64])

In [34]:
# bring channels to the first dimension
# so that theimages have shape of (C,N,H,W)
theimages=images.permute(1,0,2,3)

In [35]:
# let's confirm that theimages has expected shape
theimages.shape

torch.Size([3, 6552, 64, 64])

In [36]:
# calculate mean across channels
theimages.mean(dim=(1,2,3))

tensor([0.4353, 0.3773, 0.2871])

In [37]:
# calculate standard deviation across channels
theimages.std(dim=(1,2,3))

tensor([0.2856, 0.2333, 0.2585])